# Colab setup

In [1]:
if 'google.colab' in str(get_ipython()):
    ! {sys.executable} -m pip install pytorch-lifestream

## Setup

In [2]:
import torch

In [3]:
%load_ext pyinstrument

## Data preproccessing

In [4]:
import pandas as pd

source_data = pd.read_csv('https://huggingface.co/datasets/dllllb/age-group-prediction/resolve/main/transactions_train.csv.gz?download=true', compression='gzip')
source_data.head(2)

,client_id,trans_date,small_group,amount_rur
0,33172,6,4,71.463
1,33172,6,35,45.017


In [5]:
from ptls.preprocessing.pandas.pandas_preprocessor import PandasDataPreprocessor

preprocessor = PandasDataPreprocessor(
    col_id='client_id',
    col_event_time='trans_date',
    event_time_transformation='none',
    cols_category=['small_group'],
    cols_numerical=['amount_rur'],
    return_records=True,
)

c:\users\user\appdata\local\programs\python\python39\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
c:\users\user\appdata\local\programs\python\python39\lib\site-packages\dask\dataframe\__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


Creating Dask Server
Link Dask Server - http://10.64.4.227:8787/status


In [6]:
%%pyinstrument
dataset = preprocessor.fit_transform(source_data)

D:\WORK\Repo\pytorch-lifestream_fork\ptls\preprocessing\pandas\pandas_transformation\category_identity_encoder.py:62: FutureWarning: The provided callable <built-in function min> is currently using Series.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  self.min_fit_index, self.max_fit_index = x.astype(int).agg([min, max])
D:\WORK\Repo\pytorch-lifestream_fork\ptls\preprocessing\pandas\pandas_transformation\category_identity_encoder.py:62: FutureWarning: The provided callable <built-in function max> is currently using Series.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  self.min_fit_index, self.max_fit_index = x.astype(int).agg([min, max])
D:\WORK\Repo\pytorch-lifestream_fork\ptls\preprocessing\pandas\pandas_transformation\category_identity_encoder.py:66: UserWarning: 0 values fount in small_group. 0 is a p

AttributeError: 'DataFrameGroupBy' object has no attribute 'to_dict'

_     ._   __/__   _ _  _  _ _/_   Recorded: 18:16:58  Samples:  132
 /_//_/// /_\ / //_// / //_'/ //     Duration: 274.216   CPU time: 227.141
/   _/                      v4.6.2

Program: c:\users\user\appdata\local\programs\python\python39\lib\site-packages\ipykernel_launcher.py -f C:\Users\user\AppData\Roaming\jupyter\runtime\kernel-ef9995f6-414d-4efa-928c-9eac2373632c.json

274.216 <module>  ..\..\ipykernel_15700\2776624806.py:2
`- 274.216 PandasDataPreprocessor.wrapped  sklearn\utils\_set_output.py:138
   `- 274.216 PandasDataPreprocessor.fit_transform  ptls\preprocessing\base\data_preprocessor.py:104
      `- 274.215 Maybe.then  pymonad\monad.py:132
            [2 frames hidden]  pymonad
               274.211 Maybe.map  pymonad\maybe.py:60
               `- 273.020 <lambda>  ptls\preprocessing\base\data_preprocessor.py:109
                  `- 273.019 PandasDataPreprocessor._apply_aggregation  ptls\preprocessing\base\data_preprocessor.py:88
                     `- 272.849 DaskDispatcher.evaluate  ptls\preprocessing\multithread_dispatcher.py:40
                        `- 272.848 Maybe.then  pymonad\monad.py:132
                              [2 frames hidden]  pymonad
                                 272.848 Maybe.map  pymonad\maybe.py:60
                                 `- 272.848 <lambda>  ptls\preprocessing\multithread_dispatcher.py:44
                                    `- 272.848 DaskDispatcher._multithread_eval  ptls\preprocessing\multithread_dispatcher.py:22
                                       `- 272.848 CallableObjectWrapper.__call__  joblib\externals\loky\cloudpickle_wrapper.py:31
                                          `- 272.847 DaskDispatcher.evaluate_single  ptls\preprocessing\multithread_dispatcher.py:49
                                             `- 272.847 UserGroupTransformer.wrapped  sklearn\utils\_set_output.py:138
                                                   [3 frames hidden]  sklearn
                                                      272.847 UserGroupTransformer.wrapped  sklearn\utils\_set_output.py:138
                                                      `- 272.847 UserGroupTransformer.transform  ptls\preprocessing\base\transformation\user_group_transformer.py:72
                                                         |- 217.628 UserGroupTransformer.df_to_feature_arrays  ptls\preprocessing\base\transformation\user_group_transformer.py:66
                                                         |- 48.251 [self]  ptls\preprocessing\base\transformation\user_group_transformer.py
                                                         `- 5.210 DataFrame.sort_index  pandas\core\frame.py:7040
                                                               [6 frames hidden]  pandas

In [7]:
import pickle
with open('preprocessor.p', 'wb') as f:
    pickle.dump(preprocessor, f)

In [8]:
dataset = sorted(dataset, key=lambda x: x['client_id'])

In [9]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(dataset, test_size=0.2, random_state=42)

len(train), len(test)

(24000, 6000)

In [10]:
train[0].keys()

dict_keys(['client_id', 'trans_date', 'event_time', 'small_group', 'amount_rur'])

## Embedding training

Model training in our framework organised via pytorch-lightning (pl) framework.
The key parts of neural networks training in pl are: 

    * model (`pytorch_lightning.LightningModule`)
    * data loader (`torch.utils.data.DataLoader`)
    * trainer (`pytorch_lightning.Trainer`)
    
For futher details check https://pytorchlightning.ai/

### Model definition

In [11]:
from functools import partial
from ptls.nn import TrxEncoder, RnnSeqEncoder
from ptls.frames.coles import CoLESModule

trx_encoder_params = dict(
    embeddings_noise=0.003,
    numeric_values={'amount_rur': 'identity'},
    embeddings={
        'trans_date': {'in': 800, 'out': 16},
        'small_group': {'in': 250, 'out': 16},
    },
)

seq_encoder = RnnSeqEncoder(
    trx_encoder=TrxEncoder(**trx_encoder_params),
    hidden_size=256,
    type='gru',
)

model = CoLESModule(
    seq_encoder=seq_encoder,
    optimizer_partial=partial(torch.optim.Adam, lr=0.001),
    lr_scheduler_partial=partial(torch.optim.lr_scheduler.StepLR, step_size=30, gamma=0.9),
)

### Data loader

In [ ]:
from ptls.data_load.datasets import MemoryMapDataset
from ptls.data_load.iterable_processing import SeqLenFilter
from ptls.frames.coles import ColesDataset
from ptls.frames.coles.split_strategy import SampleSlices
from ptls.frames import PtlsDataModule

In [12]:
in_memory_dataset = MemoryMapDataset(data=train,i_filters=[SeqLenFilter(min_seq_len=25)])

In [ ]:
data_splitter = SampleSlices(split_count=5,cnt_min=25,cnt_max=200)

In [ ]:
coles_df = ColesDataset(data=in_memory_dataset,splitter=data_splitter)

In [ ]:
train_dl = PtlsDataModule(
    train_data=coles_df,
    train_num_workers=16,
    train_batch_size=256,
)

### Trainer

In [13]:
import torch
import pytorch_lightning as pl


trainer = pl.Trainer(
    max_epochs=15,
    accelerator="cuda" if torch.cuda.is_available() else "cpu",
    devices=1 if torch.cuda.is_available() else "auto",
    enable_progress_bar=False,
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


### Training 

In [ ]:
%%pyinstrument
print(f'logger.version = {trainer.logger.version}')
trainer.fit(model, train_dl)
print(trainer.logged_metrics)

Missing logger folder: D:\WORK\Repo\pytorch-lifestream\demo\lightning_logs
c:\users\user\appdata\local\programs\python\python39\lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.

  | Name               | Type            | Params | Mode 
---------------------------------------------------------------
0 | _loss              | ContrastiveLoss | 0      | train
1 | _seq_encoder       | RnnSeqEncoder   | 240 K  | train
2 | _validation_metric | BatchRecallTopK | 0      | train
3 | _head              | Head            | 0      | train
---------------------------------------------------------------
240 K     Trainable params
0         Non-trainable params
240 K     Total params
0.962     Total estimated model params size (MB)


logger.version = 0


c:\users\user\appdata\local\programs\python\python39\lib\site-packages\torch\utils\data\dataloader.py:561: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
c:\users\user\appdata\local\programs\python\python39\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:419: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


### Save sequence encoder for other experiments

In [14]:
torch.save(seq_encoder.state_dict(), "coles-emb.pt")

## Inference 

In [15]:
# embedding inference
%%pyinstrument
from ptls.data_load.datasets import inference_data_loader

train_dl = inference_data_loader(train, num_workers=0, batch_size=256)
train_embeds = torch.vstack(trainer.predict(model, train_dl, ))

test_dl = inference_data_loader(test, num_workers=0, batch_size=256)
test_embeds = torch.vstack(trainer.predict(model, test_dl))

train_embeds.shape, test_embeds.shape

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/home/kireev/pipenv_envs/pytorch-lifestream-KxQJF1XF/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/kireev/pipenv_envs/pytorch-lifestream-KxQJF1XF/lib/python3.8/site-packages/pytorch_lightning/loops/epoch/prediction_epoch_loop.py:175: UserWarning: Lightning couldn't infer the indices fetched for your dataloader.
  warning_cache.warn("Lightning couldn't infer the indices fetched for your dataloader.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


(torch.Size([24000, 256]), torch.Size([6000, 256]))

In [16]:
# join target and embeddings

df_target = pd.read_csv('https://huggingface.co/datasets/dllllb/age-group-prediction/resolve/main/train_target.csv?download=true')
df_target = df_target.set_index('client_id')
df_target.rename(columns={"bins": "target"}, inplace=True)

train_df = pd.DataFrame(data=train_embeds, columns=[f'embed_{i}' for i in range(train_embeds.shape[1])])
train_df['client_id'] = [x['client_id'] for x in train]
train_df = train_df.merge(df_target, how='left', on='client_id')

test_df = pd.DataFrame(data=test_embeds, columns=[f'embed_{i}' for i in range(test_embeds.shape[1])])
test_df['client_id'] = [x['client_id'] for x in test]
test_df = test_df.merge(df_target, how='left', on='client_id')

print(train_df.shape, test_df.shape)

(24000, 258) (6000, 258)


Obtained embeddings can be used as features for model training

For example:

In [17]:
from sklearn.ensemble import RandomForestClassifier

embed_columns = [x for x in train_df.columns if x.startswith('embed')]
x_train, y_train = train_df[embed_columns], train_df['target']
x_test, y_test = test_df[embed_columns], test_df['target']

clf = RandomForestClassifier()
clf.fit(x_train, y_train)
clf.score(x_test, y_test)

0.5756666666666667